這份 Notebook 示範用 OpenAI Function Calling 來達成 Agent 效果，將 colab 702 的寫法包裹更好，可以遞迴執行工具

In [ ]:
from google.colab import userdata
openai_api_key = userdata.get('openai_api_key')

In [ ]:
import requests
import json
from pprint import pp

In [ ]:
def get_completion(messages, model="gpt-3.5-turbo", temperature=0, max_tokens=1000, tools=None, tool_choice=None):
  payload = { "model": model, "temperature": temperature, "messages": messages, "max_tokens": max_tokens }
  if tools:
    payload["tools"] = tools
  if tool_choice:
    payload["tool_choice"] = tool_choice

  headers = { "Authorization": f'Bearer {openai_api_key}', "Content-Type": "application/json" }
  response = requests.post('https://api.openai.com/v1/chat/completions', headers = headers, data = json.dumps(payload) )
  obj = json.loads(response.text)

  if response.status_code == 200 :
    return obj["choices"][0]["message"] # 改成回傳上一層 message 物件
  else :
    return obj["error"]

In [ ]:
!pip install googlesearch-python

  Preparing metadata (setup.py) ... done
  Created wheel for googlesearch-python: filename=googlesearch_python-1.2.3-py3-none-any.whl size=4209 sha256=759efc5740f1b67651b6a09ff3911bdfb78baf816a500e0e8938395762c0642b
  Stored in directory: /root/.cache/pip/wheels/98/24/e9/6c225502948c629b01cc895f86406819281ef0da385f3eb669
Successfully built googlesearch-python


## 我們再次用 googlesearch 工具可查詢最新消息

如果你是做 RAG 應用，這個工具可以換成你的知識 Retriever 工具

In [ ]:
from googlesearch import search

In [ ]:
def google_search(keyword):
  content = ""
  for item in search(keyword, advanced=True, num_results=3, lang='zh-tw'):
    content += f"Title: {item.title}\n Description: {item.description}\n\n"
  return content

## 用 Function Calling 做 Agent

一個問題可能需要多次執行 functions，才能夠解決。OpenAI 可能不只一次叫我們呼叫 function。

用 function calling 就可以做出 agent，效果和 ReACT 一樣。

我們來把上一個 notebook 的 function calling 步驟，改成會執行工具的遞迴 function

In [ ]:
available_tools = {
  "google_search": google_search,
}

def get_completion_with_function_execution(messages, model="gpt-3.5-turbo", temperature=0, max_tokens=300, tools=None, tool_choice=None):
  print(f"called prompt: {messages}")
  response = get_completion(messages, model=model, temperature=temperature, max_tokens=max_tokens, tools=tools,tool_choice=tool_choice)

  if response.get("tool_calls"): # 或用 response 裡面的 finish_reason 判斷也行
    messages.append(response)

    # ------ 呼叫函數，這裡改成執行多 tool_calls (可以改成平行處理，目前用簡單的迴圈)
    for tool_call in response["tool_calls"]:
      function_name = tool_call["function"]["name"]
      function_args = json.loads(tool_call["function"]["arguments"])
      function_to_call = available_tools[function_name]

      print(f"   called function {function_name} with {function_args}")
      function_response = function_to_call(**function_args)
      messages.append(
          {
              "tool_call_id": tool_call["id"], # 多了 toll_call_id
              "role": "tool",
              "name": function_name,
              "content": function_response,
          }
      )

    # 進行遞迴呼叫
    return get_completion_with_function_execution(messages, model=model, temperature=temperature, max_tokens=max_tokens, tools=tools,tool_choice=tool_choice)

  else:
    return response # response["content"]

In [ ]:
messages = [{"role": "user", "content": "今天台北天氣如何?"}]

tools = [
    {
        "type": "function",
        "function": {
            "name": "google_search",
            "description": "搜尋最新的資訊",
            "parameters": {
                "type": "object",
                "properties": {
                    "keyword": {
                        "type": "string",
                        "description": "搜尋關鍵字",
                    }
                },
                "required": ["keyword"],
            },
        },
    }
]

response = get_completion_with_function_execution(messages, tools=tools)
pp(response)

called prompt: [{'role': 'user', 'content': '今天台北天氣如何?'}]
   called function google_search with {'keyword': '台北天氣'}
called prompt: [{'role': 'user', 'content': '今天台北天氣如何?'}, {'role': 'assistant', 'content': None, 'tool_calls': [{'id': 'call_pfdaRqZB3SH9XVXljLuq9Osx', 'type': 'function', 'function': {'name': 'google_search', 'arguments': '{"keyword":"台北天氣"}'}}]}, {'tool_call_id': 'call_pfdaRqZB3SH9XVXljLuq9Osx', 'role': 'tool', 'name': 'google_search', 'content': 'Title: 縣市預報- 臺北市\n Description: 今日白天 陰短暫雨 17 - 1863 - 64降雨機率30%稍有寒意; 今晚明晨 多雲時陰 15 - 1759 - 63降雨機率20%寒冷至稍有寒意; 明日白天 陰時多雲短暫陣雨 15 - 1959 - 66降雨\xa0...\n\nTitle: 臺北市, 台北市, 臺灣每小時天氣\n Description: RealFeel Shade™63°. 風東北11英里/小时. 空氣品質一般. 最大紫外線指數0 低. 陣風14英里/小时. 濕度69%. 露點56° F. 雲層80%. 能見度7英里. 雲冪2800英尺\xa0...\n\nTitle: 臺北市, 台灣- 氣象預報| 地圖\n Description: 星期, 天氣狀況, 降雨機率, 溫度. 檢視氣象預報詳細資訊 星期六, 陣雨. 降雨機率: 51%. 華氏高溫: 65°; 攝氏高溫: 19°; 華氏低溫: 54°; 攝氏低溫: 13°.\n\nTitle: 東榮, 臺北市10 天天氣預報\n Description: 紫外線指數5 (最大值11). 日出06:07. 日落18:00. 週日10 | 夜晚. 15°. 21%

### 挑戰難一點的問題，其實需要多次呼叫 functions

gpt-3.5-turbo-1106 之後的版本，以及 gpt-4-turbo-preview，支援 parallel function calling

https://platform.openai.com/docs/guides/function-calling/parallel-function-calling

In [ ]:
user_messages = [{"role": "user", "content": "今天台北、新竹、高雄、台中分別的天氣如何?"}]
response = get_completion_with_function_execution(user_messages, tools=tools)
pp(response)

called prompt: [{'role': 'user', 'content': '今天台北、新竹、高雄、台中分別的天氣如何?'}]
   called function google_search with {'keyword': '台北天氣'}
   called function google_search with {'keyword': '新竹天氣'}
   called function google_search with {'keyword': '高雄天氣'}
   called function google_search with {'keyword': '台中天氣'}
called prompt: [{'role': 'user', 'content': '今天台北、新竹、高雄、台中分別的天氣如何?'}, {'role': 'assistant', 'content': None, 'tool_calls': [{'id': 'call_yKp4QLtbS5SM528KRbTlh9Sl', 'type': 'function', 'function': {'name': 'google_search', 'arguments': '{"keyword": "台北天氣"}'}}, {'id': 'call_t8rfXOtA518Z5qmptxHcHjLj', 'type': 'function', 'function': {'name': 'google_search', 'arguments': '{"keyword": "新竹天氣"}'}}, {'id': 'call_xFHc0Qvg0HcKQiZUSZgTzkNr', 'type': 'function', 'function': {'name': 'google_search', 'arguments': '{"keyword": "高雄天氣"}'}}, {'id': 'call_BJCtRkmpsh6wOVszc0NIEUPj', 'type': 'function', 'function': {'name': 'google_search', 'arguments': '{"keyword": "台中天氣"}'}}]}, {'tool_call_id': 'call_yKp4Q

### 之前的版本(例如 gpt-3.5-turbo-0613 和 gpt-4-0613)，不支援 parallel function calling，只能一個一個來

其他家的 function calling 目前也還不支援

能平行的效能比起舊版循序會大大提升! 節省了多次 API 往返!

In [ ]:
user_messages = [{"role": "user", "content": "今天台北、新竹、高雄、台中分別的天氣如何?"}]
response = get_completion_with_function_execution(user_messages, tools=tools, model="gpt-3.5-turbo-0613")
pp(response)

called prompt: [{'role': 'user', 'content': '今天台北、新竹、高雄、台中分別的天氣如何?'}]
   called function google_search with {'keyword': '台北天氣'}
called prompt: [{'role': 'user', 'content': '今天台北、新竹、高雄、台中分別的天氣如何?'}, {'role': 'assistant', 'content': None, 'tool_calls': [{'id': 'call_Ed6ubMLORvNYRmaumXi1091L', 'type': 'function', 'function': {'name': 'google_search', 'arguments': '{\n  "keyword": "台北天氣"\n}'}}]}, {'tool_call_id': 'call_Ed6ubMLORvNYRmaumXi1091L', 'role': 'tool', 'name': 'google_search', 'content': 'Title: 縣市預報- 臺北市\n Description: 今日白天 晴時多雲 25 - 3077 - 86降雨機率10%舒適; 今晚明晨 多雲時晴 19 - 2566 - 77降雨機率20%稍有寒意至舒適; 明日白天 陰短暫陣雨 19 - 2266 - 72降雨機率30%稍\xa0...\n\nTitle: 臺北市, 台灣- 氣象預報| 地圖\n Description: 星期, 天氣狀況, 降雨機率, 溫度. 檢視氣象預報詳細資訊 星期三, 晴朗. 降雨機率: 0%. 華氏高溫: 83°; 攝氏高溫: 29°; 華氏低溫: 67°; 攝氏低溫: 20°.\n\nTitle: 臺北市, 台北市, 臺灣每小時天氣\n Description: RealFeel Shade™80°. 風西北偏西13英里/小时. 空氣品質不健康. 最大紫外線指數2 低. 陣風16英里/小时. 濕度68%. 露點69° F. 雲層2%. 能見度10英里. 雲冪30000英尺\xa0...\n\nTitle: 東榮, 臺北市10 天天氣預報\n Description: 週三21 | 白天. 28°. 10

## Chaining v.s Agent

* Chaining 是固定步驟、固定的 Prompts 次數來回
* Agent 沒有固定會有幾次 Prompts 來回，是讓 GPT 決定的

## 備註

1. 這個範例只有放一種工具，當然可以放多個工具
2. 可以限制迴圈次數，避免不幸一直呼叫
